In [ ]:
!pip install PyICU

In [3]:
import icu
def transliterator_from_rules(name, rules):
    fromrules = icu.Transliterator.createFromRules(name, rules)
    icu.Transliterator.registerInstance(fromrules)
    return icu.Transliterator.createInstance(name)

In [4]:
rules = r"""
# Based on Russian-Latin-BGN.xml
# Minimal filter (Russian alphabet)
::[АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя];
::NFC;

######################################################################
# Variables
######################################################################
$wordBoundary = [^[:L:][:M:][:N:]] ;
$upperVowels  = [АЕЁЭИОУЫЮЯ] ;
$lowerVowels  = [аеёэиоуыюя] ;
$vowels       = [$upperVowels $lowerVowels] ;
$upperCons    = [[:Uppercase:]-$vowels] ;
$lowerCons    = [[:Lowercase:]-$vowels] ;
$consonants   = [$upperCons $lowerCons] ;
$upper        = [:Uppercase:] ;
$lower        = [:Lowercase:] ;

# Convenience sets for contextual rules (in Cyrillic, before transliteration)
$ZSTC = [ЗзСсТтЦц] ;
$CHSH = [ЧчШшЩщЖж] ;

######################################################################
# Context-first rules (so they don't get masked by simple mappings)
######################################################################

# ---- е (Swedish: je word-initial / after vowel/й/ь/ъ; otherwise e) ----
$wordBoundary { Е } $upper → JE ;
$wordBoundary { Е } $lower → Je ;
$wordBoundary { е → je ;

[$upperVowels [ЙЪЬ]] { Е } $upper → JE ;
[$upperVowels [ЙЪЬ]] { Е } $lower → Je ;
[$upperVowels $lowerVowels [ЙйЪъЬь]] { е → je ;

# ---- ё (default jo; after з/с/т/ц -> io; after ч/ш/щ/ж -> o) ----
# Word-initial / after vowels/й/ь/ъ => "jo"
$wordBoundary { Ё } $upper → JO ;
$wordBoundary { Ё } $lower → Jo ;
$wordBoundary { ё → jo ;
[$upperVowels [ЙЪЬ]] { Ё } $upper → JO ;
[$upperVowels [ЙЪЬ]] { Ё } $lower → Jo ;
[$upperVowels $lowerVowels [ЙйЪъЬь]] { ё → jo ;

# After З/С/Т/Ц => "io"
$ZSTC { Ё } $upper → IO ;
$ZSTC { Ё } $lower → Io ;
$ZSTC { ё → io ;

# After Ч/Ш/Щ/Ж => "o"
$CHSH { Ё } $upper → O ;
$CHSH { Ё } $lower → O ;
$CHSH { ё → o ;

# Force a pass boundary before proceeding to basic letters
::Null;

# ---- ю (default ju; after з/с/т/ц -> iu; after ч/ш/щ/ж -> u) ----
$wordBoundary { Ю } $upper → JU ;
$wordBoundary { Ю } $lower → Ju ;
$wordBoundary { ю → ju ;
[$upperVowels [ЙЪЬ]] { Ю } $upper → JU ;
[$upperVowels [ЙЪЬ]] { Ю } $lower → Ju ;
[$upperVowels $lowerVowels [ЙйЪъЬь]] { ю → ju ;

$ZSTC { Ю } $upper → IU ;
$ZSTC { Ю } $lower → Iu ;
$ZSTC { ю → iu ;

$CHSH { Ю } $upper → U ;
$CHSH { Ю } $lower → U ;
$CHSH { ю → u ;

# ---- я (default ja; after з/с/т/ц -> ia; after ч/ш/щ/ж -> a) ----
$wordBoundary { Я } $upper → JA ;
$wordBoundary { Я } $lower → Ja ;
$wordBoundary { я → ja ;
[$upperVowels [ЙЪЬ]] { Я } $upper → JA ;
[$upperVowels [ЙЪЬ]] { Я } $lower → Ja ;
[$upperVowels $lowerVowels [ЙйЪъЬь]] { я → ja ;

$ZSTC { Я } $upper → IA ;
$ZSTC { Я } $lower → Ia ;
$ZSTC { я → ia ;

$CHSH { Я } $upper → A ;
$CHSH { Я } $lower → A ;
$CHSH { я → a ;

######################################################################
# Alphabetic mappings (Swedish outcomes + nice titlecasing)
######################################################################
А → A ;  а → a ;
Б → B ;  б → b ;
В → V ;  в → v ;
Г → G ;  г → g ;
Д → D ;  д → d ;

Е → E ;  е → e ;   # remaining 'e' cases fall back to 'e' (contexts handled above)
Ё → Jo ; ё → jo ;  # leftover (if any) — normally handled in contexts

Ж } $lower → Zj ;  Ж → ZJ ;  ж → zj ;
З → Z ;  з → z ;
И → I ;  и → i ;
Й → J ;  й → j ;
К → K ;  к → k ;
Л → L ;  л → l ;
М → M ;  м → m ;
Н → N ;  н → n ;
О → O ;  о → o ;
П → P ;  п → p ;
Р → R ;  р → r ;
С → S ;  с → s ;
Т → T ;  т → t ;
У → U ;  у → u ;
Ф → F ;  ф → f ;

Х } $lower → Ch ;
Х → CH ;  х → ch ;
Ц } $lower → Ts ;
Ц → TS ;
ц → ts ;
Ч } $lower → Tj ;
Ч → TJ ;
ч → tj ;
Ш } $lower → Sj ;
Ш → SJ ;
ш → sj ;
Щ } $lower → Sjtj ;
Щ → SJTJ ;
щ → sjtj ;

Ъ → ;
ъ → ;       # omitted
Ы → Y ;
ы → y ;
Ь → ;
ь → ;       # omitted

Э → E ;
э → e ;

# (Ю/Я basic forms handled via contexts above; fallbacks retained)
Ю } $lower → Ju ;
Ю → JU ;
ю → ju ;
Я } $lower → Ja ;
Я → JA ;
я → ja ;
"""

In [6]:
rusv = transliterator_from_rules("ru-sv", rules)

tests = [
    ("Москва", "Moskva"),
    ("Чайковский", "Tjajkovskij"),
    ("Щука", "Sjtjuka"),
    ("Жириновский", "Zjirinovskij"),
    ("Юрий", "Jurij"),
    ("Яковлев", "Jakovlev"),
    ("Хрущёв", "Chrusjtjov"),
    ("Циолковский", "Tsiolkovskij")
]

# --- Run test ---
for w in tests:
    assert w[1] == rusv.transliterate(w[0])
    print(f"{w[0]:15s} → {rusv.transliterate(w[0])}")


Москва          → Moskva
Чайковский      → Tjajkovskij
Щука            → Sjtjuka
Жириновский     → Zjirinovskij
Юрий            → Jurij
Яковлев         → Jakovlev
Хрущёв          → Chrusjtjov
Циолковский     → Tsiolkovskij


In [49]:
fr_rules = r"""
::[АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя];
::NFC;

$wordBoundary = [^[:L:][:M:][:N:]];
$upperVowels  = [АЕЁЭИОУЫЮЯ];
$lowerVowels  = [аеёэиоуыюя];
$vowels       = [$upperVowels $lowerVowels];
$upper        = [:Uppercase:];
$lower        = [:Lowercase:];
$ZSTC         = [ЗзСсТтЦц];
$CHSH         = [ЧчШшЩщЖж];

$IJ           = [ИиЙй];
$Front        = [ЕеИиЫыЭэ];
$FrontUpper   = [ЕИЫЭ];
$FrontLower   = [еиыэ];

$latinVowels  = [AEIOUYaeiouyÄËÏÖÜŸäëïöüÿ];
$latIJ        = [Ïï];

# -----------------------
# Е (E) – placement & case
# -----------------------

# Word-initial E → Ie / IE (titlecased by next-char case)
$wordBoundary { Е } $lower    → Ie;
$wordBoundary { Е } [^$lower] → IE;
$wordBoundary { е }           → ie;

# If a Cyrillic or Latin vowel is to the left AND next is Й/й, keep plain e (no diaeresis before й)
[ ($vowels-$IJ) | ($latinVowels-$latIJ) ] { Е } $IJ → E;
[ ($vowels-$IJ) | ($latinVowels-$latIJ) ] { е } $IJ → e;

# Don’t create “ïe” after Cyrillic/Latin G/Gu (Gueorgui, not Guïe…)
[Гг] { Е } → E;
[Гг] { е } → e;
[Gg] { Е } → E;
[Gg] { е } → e;
Gu { Е } → e;
Gu { е } → e;
GU { Е } → E;
GU { е } → E;

# After vowel (Cyrillic or Latin but not from И/Й), insert diaeresis: …ïe / …ÏE
[ ($vowels-$IJ) | ($latinVowels-$latIJ) ] { Е } $lower    → ïe;
[ ($vowels-$IJ) | ($latinVowels-$latIJ) ] { Е } [^$lower] → ÏE;
[ ($vowels-$IJ) | ($latinVowels-$latIJ) ] { е }           → ïe;

# After И/Й or ь/ъ (or Latin Ï/ï), use ie with titlecasing by next-char case
[ $IJ ЬьЪъ $latIJ ] { Е } $lower    → Ie;
[ $IJ ЬьЪъ $latIJ ] { Е } [^$lower] → IE;
[ $IJ ЬьЪъ $latIJ ] { е }           → ie;

# -----------------------
# И (I) – diaeresis after vowel (≠ И)
# -----------------------
[$vowels-[Ии]] { И } → Ï;
[$vowels-[Ии]] { и } → ï;

# -----------------------
# Й (JOT) – finals and special clusters
# -----------------------

# Suppress й in the very common …иев / …ьев cluster (Dmitriev, not Dmitriïev)
[ИиЬь] { Й } [Ее] → ;

# Final -ий / -ый endings at end of word
ий $ → i;
Ий $ → i;
ый $ → y;
Ый $ → y;

# Elsewhere, й → ï
Й → Ï;
й → ï;

# -----------------------
# Ю (JU) – contexts + casing by right char
# -----------------------
$IJ { Ю } $lower    → Ou;
$IJ { Ю } [^$lower] → OU;
$IJ { ю }           → ou;

[$vowels-$IJ] { Ю } $lower    → Ïou;
[$vowels-$IJ] { Ю } [^$lower] → ÏOU;
[$vowels-$IJ] { ю }           → ïou;

Ю } $lower    → Iou;
Ю } [^$lower] → IOU;
ю             → iou;

# -----------------------
# Я (JA) – contexts + casing by right char
# -----------------------
$IJ { Я } $lower    → A;
$IJ { Я } [^$lower] → A;
$IJ { я }           → a;

[$vowels-$IJ] { Я } $lower    → Ïa;
[$vowels-$IJ] { Я } [^$lower] → ÏA;
[$vowels-$IJ] { я }           → ïa;

Я } $lower    → Ia;
Я } [^$lower] → IA;
я             → ia;

# -----------------------
# Ё (JO) – optional o after sibilants, else Io/io with casing
# -----------------------
$CHSH { Ё } → O;
$CHSH { ё } → o;

Ё } $lower    → Io;
Ё } [^$lower] → IO;
ё             → io;

# -----------------------
# Intervocalic С → ss
# -----------------------
$vowels { С } $vowels → ss;
$vowels { с } $vowels → ss;

# -----------------------
# Г before front vowel → Gu / GU / gu
# -----------------------
Г } $FrontLower → Gu;
Г } $FrontUpper → GU;
г } $FrontLower → gu;
г } $FrontUpper → gu;

# -----------------------
# Lowercase-only final -ин / -ын → -ine (no effect on ALL-CAPS)
# -----------------------
[иы] { н } $ → ne;
[иы] { н } $wordBoundary → ne;

# -----------------------
# Simple letters
# -----------------------
А → A;
а → a;
Б → B;
б → b;
В → V;
в → v;
Г → G;
г → g;
Д → D;
д → d;
Е → E;
е → e;
Ж → J;
ж → j;
З → Z;
з → z;
И → I;
и → i;
К → K;
к → k;
Л → L;
л → l;
М → M;
м → m;
Н → N;
н → n;
О → O;
о → o;
П → P;
п → p;
Р → R;
р → r;
С → S;
с → s;
Т → T;
т → t;

# -----------------------
# Digraphs/trigraphs with right-case titlecasing
# -----------------------
У } $lower    → Ou;
У } [^$lower] → OU;
у             → ou;

Х } $lower    → Kh;
Х } [^$lower] → KH;
х             → kh;

Ц } $lower    → Ts;
Ц } [^$lower] → TS;
ц             → ts;

Ч } $lower    → Tch;
Ч } [^$lower] → TCH;
ч             → tch;

Ш } $lower    → Ch;
Ш } [^$lower] → CH;
ш             → ch;

Щ } $lower    → Chtch;
Щ } [^$lower] → CHTCH;
щ             → chtch;

Ъ → ;
ъ → ;
Ы → Y;
ы → y;
Ь → ;
ь → ;
Э → E;
э → e;

::NFC;
"""

In [54]:
fr_examples = """
Владимир → Vladimir
Борис → Boris
Смирнов → Smirnov
Сергей → Sergueï
Георгий → Gueorgui
Новгород → Novgorod
Менделеев → Mendeleïev
Чехов → Tchekhov
Дмитриев → Dmitriev
Ельцин → Eltsine
Дудаев → Doudaïev
Екатеринбург → Iekaterinbourg
Васильев → Vassiliev
Тургенев → Tourgueniev
Пётр → Piotr
Королёв → Koroliov
Горбачёв → Gorbatchev
Нижний → Nijni
Казимир → Kazimir
Михаил → Mikhaïl
Мир → Mir
Достоевский → Dostoïevski
Грозный → Grozny
Алексей → Alekseï
Андрей → Andreï
Александр → Aleksandr
Калининград → Kaliningrad
Малевич → Malevitch
Дума → Douma
Гагарин → Gagarine
Солженицын → Soljenitsyne
Магадан → Magadan
Байконур → Baïkonour
Волга → Volga
Спутник → Spoutnik
Самара → Samara
Новосибирск → Novossibirsk
Курск → Koursk
Владивосток → Vladivostok
Ульянов → Oulianov
Туполев → Tupolev
Прокофьев → Prokofiev
Михаил → Mikhaïl
Хабаровск → Khabarovsk
Цветаева → Tsvetaïeva
Черненко → Tchernenko
Пушкин → Pouchkine
Щедрин → Chtchedrine
Черномырдин → Tchernomyrdine
Область → Oblast
Элиста → Elista
Биюлин → Biouline
Нефтеюганск → Nefteïougansk
Юрий → Iouri
Союз → Soyouz
Мария → Maria
Майя → Maïa
Маяковский → Maïakovski
Ярославль → Iaroslavl
Ялта → Yalta
"""

In [55]:
tests = []
for line in fr_examples.split("\n"):
    line = line.strip()
    if line == "":
        continue
    parts = line.split(" → ")
    tests.append(parts)

In [56]:
rufr = transliterator_from_rules("ru-fr", fr_rules)

for w in tests:
    # assert w[1] == rufr.transliterate(w[0]), f"{w[0]:15s} → {rufr.transliterate(w[0])}"
    if w[1] != rufr.transliterate(w[0]):
        print("Error", f"{w[0]:15s} → {rufr.transliterate(w[0])} ({w[1]})")
    # print(f"{w[0]:15s} → {rufr.transliterate(w[0])}")

Error Дмитриев        → Dmitriïev (Dmitriev)
Error Ельцин          → Ieltsin (Eltsine)
Error Васильев        → Vasilev (Vassiliev)
Error Тургенев        → Tourguïenev (Tourgueniev)
Error Горбачёв        → Gorbatchiov (Gorbatchev)
Error Михаил          → Mikhail (Mikhaïl)
Error Гагарин         → Gagarin (Gagarine)
Error Солженицын      → Soljenitsyn (Soljenitsyne)
Error Новосибирск     → Novosibirsk (Novossibirsk)
Error Туполев         → Toupolev (Tupolev)
Error Прокофьев       → Prokoфev (Prokofiev)
Error Михаил          → Mikhail (Mikhaïl)
Error Пушкин          → Pouchkin (Pouchkine)
Error Щедрин          → Chtchedrin (Chtchedrine)
Error Черномырдин     → Tchernomyrdin (Tchernomyrdine)
Error Биюлин          → Biioulin (Biouline)
Error Нефтеюганск     → Neфteiougansk (Nefteïougansk)
Error Союз            → Soiouz (Soyouz)
Error Мария           → Mariia (Maria)
Error Майя            → Maïia (Maïa)
Error Маяковский      → Maiakovski (Maïakovski)
Error Ялта            → Ialta (Yalta)
